In [3]:
from math import sin, cos,asin, degrees,sqrt, atan2, radians
import json
import pandas as pd
import numpy as np
import csv
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches

In [4]:
bicimad_df = pd.read_csv('bicimad_stations.csv', sep=";", header=0)
emt_df =  pd.read_csv('line_headers_info.csv', sep=";", header=0)

### 4.1.1 Distancia haversine

In [6]:
def calcular_distancia(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2( sqrt(a), sqrt(1-a) )
    return R * c * 1000

## Procedimiento

In [7]:
def loop_distancia(latitud,longitud,radio,df):
    res = []
    for index, row in df.iterrows():
        distancia = calcular_distancia(latitud,longitud,row.latitude,row.longitude) 
        if distancia <= radio:
            res.append(row)
    return pd.DataFrame(res) 

In [8]:
def calcular_distancia_con_id_estacion(id_estacion,radio,dataset_busqueda):
    res =  bicimad_df[bicimad_df.id == id_estacion]
    if not res.empty:
        lon = res.longitude.values[0]
        lat = res.latitude.values[0]
        res2 = loop_distancia(res.latitude.iloc[0],res.longitude.iloc[0],radio,dataset_busqueda)
        titulo = res.name.iloc[0]
        return res2

In [15]:
def excluir_estaciones_emt(radio):
    estaciones_dentro_bicimad = []
    for index, row in bicimad_df.iterrows():
        res = calcular_distancia_con_id_estacion(row.id,radio,emt_df)
        for index, r in res.iterrows():
            r = r.tolist()
            if r not in estaciones_dentro_bicimad:
                estaciones_dentro_bicimad.append(r)
    pd.DataFrame(estaciones_dentro_bicimad, columns = ["id_line","label","type_header","id_header","name_header","latitude","longitude"]).to_csv('estaciones_dentro_bicimad.csv', sep=';', encoding='utf-8',index=False)

In [18]:
excluir_estaciones_emt(300)